In [1]:
from datasets import load_dataset, load_from_disk, Dataset
# from datasets import load_from_disk
import torch

/home/elena/miniconda/envs/emcomm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
ds_val = load_from_disk("../../../datasets/coco_val_features_resnet_152")

In [3]:
ds_val

Dataset({
    features: ['coco_url', 'captions', 'image_id', 'features'],
    num_rows: 5000
})

In [5]:
checkpoint_path = "/home/elena/emcomm/emcomm_captions/checkpoints/full_game/best_epoch_3.pt"
checkpoint = torch.load(checkpoint_path, weights_only=False)


In [17]:
from collections import namedtuple

checkpoint['opts'].keys()
OptsNamedTuple = namedtuple('Opts', checkpoint['opts'].keys())
opts = OptsNamedTuple(*checkpoint['opts'].values())
opts

Opts(perceptual_dimensions=[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 

In [18]:
from features import VectorsLoader

In [19]:
# opts = checkpoint['opts']
# common_opts = checkpoint['common_opts']
data_loader = VectorsLoader(
        perceptual_dimensions=opts.perceptual_dimensions,
        n_distractors=opts.n_distractors,
        batch_size=opts.batch_size,
        train_samples=opts.train_samples,
        validation_samples=opts.validation_samples,
        test_samples=opts.test_samples,
        shuffle_train_data=opts.shuffle_train_data,
        dump_data_folder=opts.dump_data_folder,
        load_data_path=opts.load_data_path,
        seed=opts.data_seed,
    )

In [24]:
from archs import Sender, Receiver
import egg.core as core
import torch.nn.functional as F

In [25]:
def loss(
    _sender_input, _message, _receiver_input, receiver_output, _labels, _aux_input
):
    acc = (receiver_output.argmax(dim=1) == _labels).detach().float()
    loss = F.cross_entropy(receiver_output, _labels, reduction="none")
    return loss, {"acc": acc}


In [49]:
sender_orig = Sender(n_features=data_loader.n_features, n_hidden=opts.sender_hidden)

receiver = Receiver(
    n_features=data_loader.n_features, linear_units=opts.receiver_hidden
)

if opts.mode.lower() == "gs":
    sender = core.RnnSenderGS(
        sender_orig,
        opts.vocab_size,
        opts.sender_embedding,
        opts.sender_hidden,
        cell=opts.sender_cell,
        max_len=opts.max_len,
        temperature=opts.temperature,
    )

    receiver = core.RnnReceiverGS(
        receiver,
        opts.vocab_size,
        opts.receiver_embedding,
        opts.receiver_hidden,
        cell=opts.receiver_cell,
    )

    game = core.SenderReceiverRnnGS(sender, receiver, loss)
else:
    raise NotImplementedError(f"Unknown training mode, {opts.mode}")

In [50]:
test_feat = ds_val['features'][0]
test_feat

[-2.4849164485931396,
 -1.8609575033187866,
 -0.8572610020637512,
 -3.2787601947784424,
 -0.46256136894226074,
 -0.9316834211349487,
 -0.6684622764587402,
 -0.11642013490200043,
 -1.2542853355407715,
 -2.591439723968506,
 -1.4111003875732422,
 -3.214085340499878,
 -1.6102299690246582,
 -1.059586524963379,
 -4.116660118103027,
 -1.669549584388733,
 -1.8512635231018066,
 -2.442902088165283,
 -2.0985121726989746,
 -0.5741544961929321,
 -1.200879454612732,
 -3.3121471405029297,
 -3.8707706928253174,
 -1.775404691696167,
 -2.9672250747680664,
 -2.295726776123047,
 -4.336905479431152,
 -3.39382004737854,
 -1.355884075164795,
 -2.8329572677612305,
 -1.7735704183578491,
 -3.080526113510132,
 -4.84385871887207,
 -0.9667870998382568,
 -0.013195216655731201,
 -0.342542439699173,
 1.1143138408660889,
 0.16926643252372742,
 -3.2690470218658447,
 0.30506038665771484,
 -3.2891454696655273,
 -2.7209346294403076,
 -2.017390489578247,
 -0.8689264059066772,
 -3.476243019104004,
 -0.9628586173057556,
 -3.

In [51]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [52]:
sender.to(device)
test_feat = torch.tensor(test_feat).to(device)
test_feat = test_feat.unsqueeze(0)  # add batch dimension
test_feat.shape

torch.Size([1, 1000])

In [53]:
sender(test_feat).shape

torch.Size([1, 101, 70])

In [ ]:
message = sender(test_feat)#.squeeze(0)  # remove batch dimension
message_argmax = message.argmax(dim=-1)
message_argmax

tensor([[65, 16, 23, 68, 62, 50, 43,  0, 40, 66, 36, 51, 38, 42, 16, 60, 28, 32,
         43, 67, 65, 69, 69, 18,  1, 16, 59, 22, 51, 35, 34, 13, 48, 10, 34, 11,
         45, 64, 45, 21, 56, 41, 46, 23, 47,  7, 60, 62, 29, 43, 28, 27, 40,  7,
         34, 16, 49,  7, 28, 39,  9, 21, 18, 54,  7, 69, 64, 66, 60, 68,  5, 66,
          1,  2, 36, 53,  3, 40,  3, 43, 53, 38,  7, 64, 37, 32, 22, 64, 34, 20,
         42, 48, 24, 44, 34,  2, 22,  2, 48, 68,  0]], device='cuda:0')